In [1]:
# import os

# for i in range(1, 9, 1):
#     os.system(f'wget https://tatabahasabm.tripod.com/bank/set{i}.pdf')

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from tika import parser
from bs4.element import Comment
from glob import glob
import re

In [3]:
TIKA_HOST = 'http://localhost:9998'

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(file, minlen=2):
    raw_xml = parser.from_file(file, TIKA_HOST, xmlContent=True)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass
    
    return t

In [95]:
raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)

In [96]:
raw_xml

{'metadata': {'pdf:unmappedUnicodeCharsPerPage': ['0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0'],
  'pdf:PDFVersion': '1.2',
  'pdf:docinfo:title': 'Microsoft Word - ms1.doc',
  'xmp:CreatorTool': 'FinePrint pdfFactory',
  'pdf:hasXFA': 'false',
  'access_permission:modify_annotations': 'true',
  'access_permission:can_print_degraded': 'true',
  'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser',
   'org.apache.tika.parser.pdf.PDFParser'],
  'X-TIKA:content_handler': 'ToXMLContentHandler',
  'dc:creator': 'simee',
  'pdf:num3DAnnotations': '0',
  'dcterms:created': '2001-08-15T01:52:31Z',
  'dc:format': 'application/pdf; version=1.2',
  'pdf:docinfo:creator_tool': 'FinePrint pdfFactory',
  'pdf:overallPercentageUnmappedUnicodeChars': '0.0',
  'access_permission:fill_in_form': 'true',
  'pdf:hasCollection': 'false',
  'pdf:encrypted': 'false',
  'dc:title': 'Microsoft Word - ms1.doc',
  'pdf:containsNonEmbeddedFont': 'true',
  'Content-Length': '42417',
 

In [84]:
def extract_integers_from_string(input_string):
    integers = re.findall(r'\d+', input_string)
    integers = [int(integer) for integer in integers]
    return integers

In [127]:
options = ['A', 'B', 'C', 'D']

In [140]:
all_quizzes = {}
for f in sorted(glob('set*.pdf')):
    text = parse_tika(f)
    splitted = ('\n'.join(text)).split('\n')
    texts, temp = [], []
    for t in splitted:
        if 'hingga Soalan' in t or ' dan Soalan' in t or 'http://tatabahasabm.tripod.com' in t or 'soalan ini diambil dari laman' in t:
            texts.append(temp)
            temp = [t]
        else:
            temp.append(t)
    ori_temp = temp[:]
    
    answers = {}
    for t in ori_temp:
        splitted = t.split('.')
        try:
            answers[int(splitted[0])] = splitted[1].strip()
        except:
            pass
    # break
    
    quizzes = {}
    for index in range(len(texts)):
        try:
            ints = extract_integers_from_string(texts[index][0])
            min_no = min(ints)
            max_no = max(ints)
            objective = texts[index][1].strip()
            extra = []
            for no, t in enumerate(texts[index][2:]):
                if t.startswith(f'{min_no} ') or t.startswith(f'{min_no}. '):
                    break
                else:
                    extra.append(t)

            extra = '\n'.join(extra).strip()
            questions, temp = [], []
            current = min_no
            for t in texts[index][no + 2:]:
                if t.startswith(f'{current} ') or t.startswith(f'{current}. '):
                    if len(temp):
                        questions.append(temp)
                    temp = [t]
                    current += 1
                else:
                    temp.append(t)
            if len(temp):
                questions.append(temp)

            for i in range(len(questions)):
                q = '\n'.join(questions[i]).split('\n\n')
                question = q[0]
                no = question.split()[0].replace('.', '')
                question = ' '.join(question.split()[1:]).strip()
                choice = '<<>>'.join(q[1:]).split('\n')
                choice = [c.replace('<<>>', ' ').strip() for c in choice if len(c.strip())]
                if len(choice) < 4:
                    choice = '\n'.join(choice).split('   ')
                    choice = '\n'.join(choice)
                    choice = choice.split('\n')

                if len(choice) != 4:
                    continue

                choice = [f'{options[no]}. {c.strip()}' for no, c in enumerate(choice)]
                quizzes[no] = {
                    'objective': objective,
                    'context': extra,
                    'no': no,
                    'question': question.replace('�', '.'),
                    'choice': choice
                }

        except:
            pass
        
    all_quizzes[f] = {
        'quizzes': quizzes,
        'answers': answers,
    }

In [141]:
all_quizzes['set1.pdf']

{'quizzes': {'1': {'objective': 'Baca petikan di bawah, kemudian jawab soalan-soalan berikutnya',
   'context': 'Gopal : Eh, awak ingat lagi tak tentang berita manusia ditelan ular sawa tempoh hari!  \nSamat : Mestilah Sampai sekarang saya masih terbayang-bayang gambar yang tersiar \ndalam akhbar tu. \nEee............ngerinya! \nChan : Tapi yang peliknya, lepas kejadian tu tak putus-putus keluar cerita tentang ular sawa \ndalam akhbar. Ular sawa telan ayamlah, telan kambinglah, dan macam -macam lagi \nSamat : Apa yang peliknya, Chan Memanglah berita sensasi macam tu disukai oleh orang \nramai Jadi wartawan pun jalankanlah tugas mereka untuk mencari cerita -cerita sedemikian! \nGopal : Agaknya tak lama lagi ada pula berita tentang ular sawa telan gajah. Haa. masa  tu \nbarulah heboh satu dunia! Adakah awak tahu ular sawa merupakan sejenis haiwan yang \ndilindungi? \nSamat : Tahu. baru semalam Cikgu Lina beritahu \nGopal : Yalah, selama ni saya tak tahu pun yang kita boleh didakwa jika m

In [142]:
with open('tatabahasabm.tripod.com-bm-kertas1.json', 'w') as fopen:
    json.dump(all_quizzes, fopen)